In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip

--2020-12-27 05:57:04--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  23.0MB/s    in 0.2s    

2020-12-27 05:57:05 (23.0 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [ ]:
!ls

ml-100k  ml-100k.zip  sample_data  tf2_DQN_run23_epch500_prediction.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import math
import random

In [ ]:
data = pd.read_csv('tf2_DQN_run23_epch500_prediction.csv')
data

,userId,actual,predictions
0,1,"[160, 33, 265, 202, 171, 20, 117, 189, 61]","[1139, 651, 1199, 585, 135, 619, 429, 960, 605]"
1,2,"[290, 312, 292, 297, 13, 50, 280, 281, 251]","[1139, 651, 569, 429, 960, 489, 443, 1153, 1038]"
2,3,"[328, 331, 334, 343]","[1139, 651, 429, 489]"
3,4,"[288, 354, 260, 356, 294, 357, 264, 361, 303, 50]","[1139, 651, 1199, 585, 135, 619, 429, 960, 605..."
4,5,"[1, 2, 98, 363, 17]","[1139, 651, 569, 429, 568]"
...,...,...,...
938,939,"[121, 258, 993, 1190, 106, 689, 409, 476, 1054]","[1139, 444, 651, 429, 294, 489, 1456, 1051, 1165]"
939,940,"[193, 66, 289, 873, 205, 14, 655, 272, 568, 315]","[1139, 651, 429, 1259, 489, 960, 1199, 435, 62..."
940,941,"[257, 258, 993, 7, 15, 147, 117, 181, 475, 124]","[1139, 429, 651, 444, 1199, 846, 489, 1259, 48..."
941,942,"[323, 261, 423, 200, 487, 584, 427, 615, 117, ...","[1139, 1477, 651, 429, 1208, 88, 1040, 401, 91..."


In [ ]:
df = pd.DataFrame()
test_precision = pd.DataFrame()

In [ ]:
df['userId'] = data['userId']
df['actual'] = data['actual'].map(lambda x: x.lstrip('[').rstrip(']'))
df['predictions'] = data['predictions'].map(lambda x: x.lstrip('[').rstrip(']'))
df

,userId,actual,predictions
0,1,"160, 33, 265, 202, 171, 20, 117, 189, 61","1139, 651, 1199, 585, 135, 619, 429, 960, 605"
1,2,"290, 312, 292, 297, 13, 50, 280, 281, 251","1139, 651, 569, 429, 960, 489, 443, 1153, 1038"
2,3,"328, 331, 334, 343","1139, 651, 429, 489"
3,4,"288, 354, 260, 356, 294, 357, 264, 361, 303, 50","1139, 651, 1199, 585, 135, 619, 429, 960, 605, 61"
4,5,"1, 2, 98, 363, 17","1139, 651, 569, 429, 568"
...,...,...,...
938,939,"121, 258, 993, 1190, 106, 689, 409, 476, 1054","1139, 444, 651, 429, 294, 489, 1456, 1051, 1165"
939,940,"193, 66, 289, 873, 205, 14, 655, 272, 568, 315","1139, 651, 429, 1259, 489, 960, 1199, 435, 625..."
940,941,"257, 258, 993, 7, 15, 147, 117, 181, 475, 124","1139, 429, 651, 444, 1199, 846, 489, 1259, 480..."
941,942,"323, 261, 423, 200, 487, 584, 427, 615, 117, 604","1139, 1477, 651, 429, 1208, 88, 1040, 401, 91,..."


In [ ]:
split_actual = df['actual'].map(lambda x: x.split(","))
split_predictions = df['predictions'].map(lambda x: x.split(","))

In [ ]:
def convert_num(series):
  int_num = []

  for i in series:
    ll = ([int(x) for x in i])
    int_num.append(ll)

  return pd.Series(int_num)

In [ ]:
test_precision['actual'] = convert_num(split_actual)
test_precision['predictions'] = convert_num(split_predictions)
test_precision['userId'] = df['userId'] = data['userId']
test_precision = test_precision.set_index('userId')
test_precision

,actual,predictions
userId,,
1,"[160, 33, 265, 202, 171, 20, 117, 189, 61]","[1139, 651, 1199, 585, 135, 619, 429, 960, 605]"
2,"[290, 312, 292, 297, 13, 50, 280, 281, 251]","[1139, 651, 569, 429, 960, 489, 443, 1153, 1038]"
3,"[328, 331, 334, 343]","[1139, 651, 429, 489]"
4,"[288, 354, 260, 356, 294, 357, 264, 361, 303, 50]","[1139, 651, 1199, 585, 135, 619, 429, 960, 605..."
5,"[1, 2, 98, 363, 17]","[1139, 651, 569, 429, 568]"
...,...,...
939,"[121, 258, 993, 1190, 106, 689, 409, 476, 1054]","[1139, 444, 651, 429, 294, 489, 1456, 1051, 1165]"
940,"[193, 66, 289, 873, 205, 14, 655, 272, 568, 315]","[1139, 651, 429, 1259, 489, 960, 1199, 435, 62..."
941,"[257, 258, 993, 7, 15, 147, 117, 181, 475, 124]","[1139, 429, 651, 444, 1199, 846, 489, 1259, 48..."


In [ ]:
data = pd.read_csv("ml-100k/ua.base", sep='\t',names="userId,movieId,rating,timestamp".split(",")) 
# sort the timestamp ascending
u1_data = data
u1_data

,userId,movieId,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
90565,943,1047,2,875502146
90566,943,1074,4,888640250
90567,943,1188,3,888640250
90568,943,1228,3,888640275


In [ ]:
user_list = data['userId']
user_list = user_list.unique()
test_novelty = pd.DataFrame(user_list, columns = ["userId"])

movie_rated_by_user = data.groupby('movieId')['userId'].count().reset_index(name='userIdCount')
movie_rated_by_user = movie_rated_by_user.set_index('movieId')
movie_rated_by_user

,userIdCount
movieId,
1,392
2,121
3,85
4,198
5,79
...,...
1678,1
1679,1
1680,1


In [ ]:
index_list = movie_rated_by_user.index.tolist()
def find_missing(lst):
    max_num = max(lst)
    min_num = min(lst)
    num_range = range(min_num, max_num+1)
    missing_number = list(set(num_range) - set(lst))
     
    return missing_number

In [ ]:
what = find_missing(index_list)
what

[1653, 1582]

In [ ]:
for i in what:
  movie_rated_by_user.loc[i] = [0]

In [ ]:
movie_rated_by_user = movie_rated_by_user.sort_index()
movie_rated_by_user

,userIdCount
movieId,
1,392
2,121
3,85
4,198
5,79
...,...
1678,1
1679,1
1680,1


In [ ]:
item_column_name = "movieId,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western"
item = pd.read_csv("ml-100k/u.item",sep='|',names=item_column_name.split(","),encoding='latin-1')
col_to_removed = ['movie_title', 'release_date', 'video_release_date', 'IMDb_URL']
clear_item = item.drop(col_to_removed, axis=1).set_index('movieId')

# the number of the kinds of topics in the whole recommendation system.
N = len(clear_item.columns)

# sum up the genre for each movieId
genre_num = clear_item.sum(axis = 1)

# group table by index and creates a dict with lists of clear_item as values
df_dict = dict(list(clear_item.groupby(clear_item.index)))

# Gather all the genres that related to all movies respectively
movieid = []
genre_list = []

for u, v in df_dict.items():
    check = v.columns[(v == 1).any()]
    if len(check) > 0:
      movieid.append(u)
      genre_list.append(check.to_list())

d = {'movieId' : movieid, 'Genres' : genre_list}

# compile in DataFrame
genre_list_by_movieid = pd.DataFrame(d)
genre_list_by_movieid.set_index('movieId', inplace=True)
genre_list_by_movieid

,Genres
movieId,
1,"[Animation, Children, Comedy]"
2,"[Action, Adventure, Thriller]"
3,[Thriller]
4,"[Action, Comedy, Drama]"
5,"[Crime, Drama, Thriller]"
...,...
1678,[Drama]
1679,"[Romance, Thriller]"
1680,"[Drama, Romance]"


# **Preprocess data**

In [ ]:
test = pd.read_csv("ml-100k/ua.test", sep='\t',names="userId,movieId,rating,timestamp".split(",")) 
test

,userId,movieId,rating,timestamp
0,1,20,4,887431883
1,1,33,4,878542699
2,1,61,4,878542420
3,1,117,3,874965739
4,1,155,2,878542201
...,...,...,...,...
9425,943,232,4,888639867
9426,943,356,4,888639598
9427,943,570,1,888640125
9428,943,808,4,888639868


In [ ]:
# Filter out the rating below 3 as we consider high rated items are those rating equal or above 3
indexNames = test[test['rating'] < 3 ].index
 
# Delete these row indexes from dataFrame
test.drop(indexNames , inplace=True)

In [ ]:
print("The total unique user in test set: ", len(test['userId'].unique()))
print("Minimum rating on test set: {}" .format(min(test['rating'])))
print("Maximum rating on test set: {}" .format(max(test['rating'])))

The total unique user in test set:  943
Minimum rating on test set: 3
Maximum rating on test set: 5


In [ ]:
test_data = test.copy().groupby('userId')['movieId'].agg(actual = lambda x: list(set(x)))
test_data

,actual
userId,
1,"[160, 33, 265, 202, 171, 20, 117, 189, 61]"
2,"[290, 312, 292, 297, 13, 50, 280, 281, 251]"
3,"[328, 331, 334, 343]"
4,"[288, 354, 260, 356, 294, 357, 264, 361, 303, 50]"
5,"[1, 2, 98, 363, 17]"
...,...
939,"[121, 258, 993, 1190, 106, 689, 409, 476, 1054]"
940,"[193, 66, 289, 873, 205, 14, 655, 272, 568, 315]"
941,"[257, 258, 993, 7, 15, 147, 117, 181, 475, 124]"


In [ ]:
# Filter out the rating below 3 since we consider only high rated items those rating equal or above 3
indexNames = test[test['rating'] < 3 ].index
 
# Delete these row indexes from dataFrame
test.drop(indexNames , inplace=True)

In [ ]:
"""
Precision computation
"""

def precision_per_user(user):
  # get the actual data
  check_a = test_precision.loc[user]
  # check intersect item
  result = float(len(np.intersect1d(check_a['actual'],check_a['predictions'])))

  # user set in test set
  # U = 
  # length of recommendation list
  L = len(check_a['predictions'])
  
  precision = (result/L)

  return precision

"""
Novelty computation
"""

def novelty_per_user(user):

  M = 943
  sum_novelty = 0
  movie_list = test_precision['predictions'].loc[user]
  for u in movie_list:
    rate_number = movie_rated_by_user.loc[u]
    if (int(rate_number) == 0):
      sum_novelty += 0
    else:
      sum_novelty += math.log2(M/rate_number)

  novelty = sum_novelty/len(movie_list)
  return novelty

"""
Diversity computation
"""

# this function is to check how many genre (topics) included in the movie item
def check_genre_num(movieid):
  n_genre = genre_num.loc[movieid]
  return n_genre

# this function is to get the list of genre that included in the movieId
def check_genre_list(movieid):
  movie_genres = genre_list_by_movieid['Genres'].loc[movieid]
  return movie_genres

# get the movie list per user and return the total genres for that list, z_Lu
def total_genre_per_list(user_movies_list):
  
  total_num_list = []
  
  for n in user_movies_list:
      genre = check_genre_list(n)
      for g in genre:
        total_num_list.append(g)
  
  return total_num_list, len(total_num_list)

# get the movie list per user and return the unique genres for that list, S_Lu
def get_unique_genre_list_by_user(user_movies_list):

  total_list = []
  
  for h in user_movies_list:
    g = check_genre_list(h)
    for k in g:
      total_list.append(k)
  
  unique_list = list(set(total_list))

  return unique_list, len(unique_list)

def H_lu(lu):
  slu, n_slu = get_unique_genre_list_by_user(lu)
  zlu, n_zlu = total_genre_per_list(lu)
  H_lu = 0
  # need to find how many times that topic j is appear in Z_Lu list
  for j in slu:
    count_appear = zlu.count(j)
    qj = count_appear/len(zlu)
    H_lu += -(qj * math.log(qj))

  return H_lu

def Div_lu(Lu):
  sl , SLu = get_unique_genre_list_by_user(Lu)
  Nt = N
  hlu = H_lu(Lu)
  Div_lu = SLu / Nt * hlu

  return Div_lu

# get the userId list in prediction result
def diversity_per_user(user):
  sum_part = 0
  movie_list = test_precision['predictions'].loc[user]

  # total number of topics in recommendation list per user
  zlu, n_zlu = total_genre_per_list(movie_list)

  for item in movie_list:
    txi = check_genre_num(item)
    value1 = txi/n_zlu
    value2 = math.log(txi/n_zlu)
    sum_part += value1*value2
  diversity_lu = -(sum_part) * Div_lu(movie_list)

  return diversity_lu

In [ ]:
precision = []
novelty = []
diversity = []

for ul in test_precision.index:
  precision.append(precision_per_user(ul))
  # p = np.mean(precision)
  # r_precision = 1/(0.05 + np.exp(-p))

  novelty.append(novelty_per_user(ul))
  # n = np.mean(novelty)
  # r_novelty = 1 - n

  diversity.append(diversity_per_user(ul))
  # d = np.mean(diversity)
  # r_diversity = 1/(0.05 + np.exp(-d))

# all_avg_precision.append(p)
# all_avg_novelty.append(n)
# all_avg_diversity.append(d)
# reward_precision.append(r_precision)
# reward_novelty.append(r_novelty)
# reward_diversity.append(r_diversity)

print("---------Precision-------")
print("Precision max: ", max(precision))
print("Precision min: ", min(precision))
print("Precision mean: ", np.mean(precision))
print("---------Novelty-------")
print("Novelty max: ", max(novelty))
print("Novelty min: ", min(novelty))
print("Novelty mean: ", np.mean(novelty))
print("---------Diversity-------")
print("Diversity max: ", max(diversity))
print("Diversity min: ", min(diversity))
print("Diversity mean: ", np.mean(diversity))

---------Precision-------
Precision max:  0.2
Precision min:  0.0
Precision mean:  0.004483327441970072
---------Novelty-------
Novelty max:  7.35891690099587
Novelty min:  3.0534316103773613
Novelty mean:  4.84698695041697
---------Diversity-------
Diversity max:  4.160124558682333
Diversity min:  -0.0
Diversity mean:  2.3024426510828695
